# Transformer
学习Transformer首先一定要看的是论文《Attention is all you need》，讲得非常清楚。

废话不多说，来看Transformer的基本结构图。这图必须必须背得滚瓜烂熟，每一个细节都需要掌握。

<img src="imgs/transformer_framework.png" width="29.99%">
<img src="imgs/transformer_framework_en.png" width="25%">

这就是经典的Encoder-Decoder结构。需要强调的一点是**图中所画的是Transformer中的一个Encoder层和一个Decoder层是如何工作的，真正的Transformer是由多个这样的Encoder层和Decoder层堆叠起来的。**英文版的图中就有清楚的画出来。

论文原话：
Encoder将输入的由$(x_1, x_2, ..., x_n)$词序列映射到一个连续的序列$z=(z_1, z_2, ..., z_n)$。给定$z$，Decoder一次生成一个词的输出序列$(y_1, y_2, ..., y_n)$。在每一个步骤中，模型都是自回归的，生成下一个词的时候都将前面生成的词作为额外的输出。

## Ecoder
根据Transformer论文中所述，Encoder部分由6个encoder层堆叠而成，每层都包含有两个子层，第一个是多头注意力机制层，第二个是简单的全连接层。并且在两个子层之后都使用残差连接，然后进行层归一化。因此一个子层的的公式为：$LayerNorm(x+Sublayer(x))$，其中$Sublayer(x)$就是一个子层的实现函数，为了方便使用残差连接，模型中的全部子层，包括Embedding层，的输出维度都为$d_{model}=512$。

## Decoder
根据Transformer论文中所述，Decoder部分也由6个decoder层堆叠而成，除了Encoder中的两个子层外，decoder还插入了第三个子层，该子层对Encoder的输出执行多头注意力机制，也就是交叉注意力机制。同样，在每一个子层后面都拼接了一个残差连接和层归一化。图中的Masked Multi-Head Attention就是掩盖了位置$i$之后的输出部分，防止decoder关注后续位置的信息，只依赖于$i$之前的信息生成输出。


## 下面实现Encoder-Decoder的框架

In [1]:
# 导包
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

/opt/conda/envs/transformer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Some convenience helper functions used throughout the notebook
def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

### 实现整体的Encoder-Decoder架构

In [12]:
class EncoderDecoder(nn.Module):
    
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder # 编码器部分
        self.decoder = decoder # 解码器部分
        self.src_embed = src_embed # 编码器输入的嵌入层Embedding
        self.tgt_embed = tgt_embed # 解码器输入的嵌入层Embedding
        self.generator = generator # 最后的线性层和softmax层

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask) # Embedding完之后才传入编码器

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask) # Embedding前面步骤生成的输出，将Encoder最后输出的key和value传入解码器
    
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1) # x维度为[batch_size, seq_len, vocab_size]，在最后一个维度上进行softmax，然后取对数
    

### 定义Encoder部分

In [13]:
# 层归一化
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [14]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)]) # 将同一个EncoderLayer拷贝N次
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers: # 逐层传入
            x = layer(x, mask)
        return x
        # return self.norm(x)

一个EncoderLayer包含两个子层，对子层进行抽象. 每一个子层的输出是$\mathrm{LayerNorm}(x + \mathrm{Sublayer}(x))$，在进行残差连接前还需要经过Dropout函数。

In [15]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return self.norm(x+ self.dropout(sublayer(x)))

每个EncoderLayer含有两个子层，一个是多头注意力机制层，一个是全连接的前馈神经网络。

In [16]:
class EncoderLayer(nn.Module):

    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(2)]) # 2个子层
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # 第一个sublayer是self-attention
        return self.sublayer[1](x, self.feed_forward) # 第二个sublayer是feed forward

### Decoder部分实现
Decoder同样是由多个DecoderLayer堆叠而成，这里的N在论文中为6.

In [17]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)]) # 将同一个DecoderLayer拷贝N次
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return x
        return self.norm(x)

In [18]:
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn # 掩码注意力
        self.src_attn = src_attn # 交叉注意力
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(3)]) # 3个子层

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask)) # 第一个sublayer是self-attention
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask)) # 第二个sublayer是交叉attention
        return self.sublayer[2](x, self.feed_forward) # 第三个sublayer是feed forward

In [19]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0

In [20]:
def example_mask():
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )


show_example(example_mask)

alt.Chart(...)

------------

# 下面对Encoder-Decoder中的各个部分进行详细解读，并手写代码实现。注意：必须先理解公式，再结合代码，加深对公式的理解。


## Attention注意力机制
Attention函数是讲一个query和一组键值对(key-value)映射到输出output,这里的$Query, Key, Value, Output$都是向量。Output输出是值Values的加权和。

对注意力机制的理解，简单来讲就是，**一句话中的一个词，只与句中的部分词有关系，而不是与每个词都有关系，attention就是衡量这种关系程度的一种方式**。如下图所示：
<p align="center">
    <img src="imgs/self_attention_mean.png" width="30%">
</p>

### Scaled Dot-Product Attention
论文中将注意力称为Scaled Dot-Product Attention，也就是缩放的点积注意力。这个注意力的计算方式为：公式的输入包含维度为$d_k$的$query$和$keys$，以及维度为$d_v$的$values$，然后将一个$query$与其他全部$key$进行点积运算，每次的计算结果除于$\sqrt{(d_k)}$，经过一次$softmax$运算后再与$values$进行点积运算。对每一个$query$都需要进行同样的操作。

计算公式为：$ Attention(Q, K, V) = softmax(\frac {QK^T}{\sqrt{d_k}})V $。 **熟记这个公式** \
Softmax计算公式为：$softmax(x)=\frac{e^{x_i}}{\sum_ie^{x_i}}$，将值压缩到$[0,1]$之间。

##### 为什么要除于$\sqrt{(d_k)}$？
论文中有说，当$d_k$较大时，点积dot-product增加得越来越大，导致经过$softmax$后，将梯度缩放到非常小的范围，也就是可能导致**梯度消失**。因此，采取除于$\sqrt{(d_k)}$的方式，来减小点积增加的幅度。这其实很好理解，$softmax$函数中，如果存在一个极大的时候，那么会导致其他较小的数在$softmax$之后趋近于0，那么这部分的梯度就可能消失。

接下来对这个过程进行图解，主要参考：[The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

#### QKV计算方式
上文所说的$query$和$keys$和$values$的计算方式如下图所示。这里以输入“Thinking Machines”为例，
<p align="center">
    <img src="imgs/attention_qkv_compute.png" width="40%">
    <img src="imgs/attention_qkv_compute2.png" width="22.28%">
</p>

注意：在**Embedding**的时候，一个token是embbeding成$d$维的向量，如果输入的token长度为4096，一个token embedding成$d_{token}=512$，那么输入层Embedding之后的维度为$[4096,512]$维，是一个矩阵。如果batch为4，那么输入层的Embedding结果就为$[4,4096,512]$。还需要注意图中的"Thinking Machines"是为了简单起见，当成一个token来处理，实际上这里还有一步tokenize的过程。

如上图所示，输入向量$X$，经过与矩阵$W^Q$相乘，得到$Query(Q)$，$K$，$V$的计算过程一样，这里的$W^Q,W^K,W^V$是模型通过训练学习到的矩阵参数。

有了$Q,K,V$之后，其中$q_1$的计算过程如下图所示。

<p align="center">
    <img src="imgs/attention_a_qkv_compute.png" width="30%">
</p>

$q_1$与$k_1,k_2$分别进行点击运算，然后除于$\sqrt{(d_k)}$，这里$d_k$就是$key$的维度，论文中的维度为64，$\sqrt{(d_k)}=8$，所以图中除的是8。**除以** $\sqrt{(d_k)}$ **可以使梯度更加稳定**。然后再经过一个$Softmax$得到向量[0.88,0.12]，最后于$value(v_1)$进行点积运算，得到$q_1$的输出$z_1$。

因此，又给Self-Attention层的公式就表示为：
<p align="center">
    <img src="imgs/self_attention_calcu_formula.png" width="30%">
    <img src="imgs/Scaled_DotProduct_Attention_compute_map.png" width="8.04%">
</p>

如果通过这两个图还是不够理解的话，建议看看这篇文章：[Understanding Self-Attention - A Step-by-Step Guide](https://armanasq.github.io/nlp/self-attention/)

------

## 下面使用代码来实现Self-Attention

In [9]:
import math
from torch import nn


class ScaleDotProductAttention(nn.Module):
    """
    计算Scale Dot-Product Attention

    q : given sentence that we focused on (decoder)
    k : every sentence to check relationship with Qeury(encoder)
    v : every sentence same with Key (encoder)
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        # input is 4 dimension tensor
        # 在输入部分，q,k,v的维度均为[batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        k_t = k.transpose(2, 3)  # 转置，k的最后两维度交换，此时k的维度为[batch_size, head, d_tensor, length]
        score = (q @ k_t) / math.sqrt(d_tensor)  # 经过计算，score的维度为[batch_size, head, length, length]

        # 2. apply masking (opt)
        if mask is not None:
            score = score.masked_fill(mask == 0, -10000) 

        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score) # 经过softmax，score的维度为[batch_size, head, length, length]

        # 4. multiply with Value
        v = score @ v # 经过矩阵乘法，v的维度为[batch_size, head, length]
        print("v:", v.shape)

        return v, score

### Multi-Head Attention 多头注意力机制
多头注意力机制，简单的讲就是**有多个不同的$Q,K,V$**。它使用了$H$组结构相同但$W^Q,W^K,W^V$参数不同的自注意力模块。输入序列首先通过不同的权重矩阵被映射到$Q,K,V$。每组查询$Query$、键$Key$和值$Value$的映射构成一个“头”，并独立地计算自注意力的运算。最后，不同头的输出被拼接在一起，并通过一个权重矩阵 $W^O∈R^{H*H}$进行映射，产生最终的输出。如
<p align="center">
    <img src="imgs/multi_head_attention.png" width="20%">
</p>
<p align="center">
    <img src="imgs/multi_head_attention_fomular.png" width="30%">
</p>

根据前面注意力公式，单个注意力的输出$z$的维度为$[d_{token}, d_v]$，也就是token数量乘以$value的维度。注意，按照论文中所述，$query,key$的维度不一定与$value$的维度相同。以前面给出的示例为例，“Thinking Machines”的经过单个注意力模块输出的结果为$z_0$，维度为：$2*3$，那么经过8个注意力模块后，就有8个维度相同的输出$z$，如下图所示：
<p align="center">
    <img src="imgs/attentions/multi_head_attention_output_z.png" width="50%">
</p>
那么这里存在的一个问题就是，全连接层无法接受多个矩阵，只能接受单个矩阵。因此，在每个注意力模块计算完成之后，再将全部计算经过进行一次拼接，再与一个权重矩阵$W_O$相乘，映射到维度为：$[d_{token}, d_v]$的矩阵。如下图所示：
<p align="center">
    <img src="imgs/attentions/multi_head_attention_output_concat.png" width="50%">
</p>

用一张图来表示多头注意力机制的计算过程如下：
<p align="center">
    <img src="imgs/attentions/multi_head_attention_output_allin.png" width="50%">
</p>


## 下面使用代码来实现Multi-Head-Attention

Multi-head attention allows the model to jointly attend to
information from different representation subspaces at different
positions. With a single attention head, averaging inhibits this.

$$
\mathrm{MultiHead}(Q, K, V) =
    \mathrm{Concat}(\mathrm{head_1}, ..., \mathrm{head_h})W^O \\
    \text{where}~\mathrm{head_i} = \mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$

Where the projections are parameter matrices $W^Q_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^K_i \in
\mathbb{R}^{d_{\text{model}} \times d_k}$, $W^V_i \in
\mathbb{R}^{d_{\text{model}} \times d_v}$ and $W^O \in
\mathbb{R}^{hd_v \times d_{\text{model}}}$.

In this work we employ $h=8$ parallel attention layers, or
heads. For each of these we use $d_k=d_v=d_{\text{model}}/h=64$. Due
to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full
dimensionality.

In [21]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = nn.ModuleList([copy.deepcopy(nn.Linear(d_model, d_model)) for _ in range(4)])
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) "Concat" using a view and apply a final linear.
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        del query
        del key
        del value
        return self.linears[-1](x)


In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [ ]:
def example_positional():
    pe = PositionalEncoding(20, 0)
    y = pe.forward(torch.zeros(1, 100, 20))

    data = pd.concat(
        [
            pd.DataFrame(
                {
                    "embedding": y[0, :, dim],
                    "dimension": dim,
                    "position": list(range(100)),
                }
            )
            for dim in [4, 5, 6, 7]
        ]
    )

    return (
        alt.Chart(data)
        .mark_line()
        .properties(width=800)
        .encode(x="position", y="embedding", color="dimension:N")
        .interactive()
    )


show_example(example_positional)

In [ ]:
import torch
print(torch.triu(torch.ones(3, 3), diagonal=0).type(
        torch.uint8
    ) == 0)


tensor([[False, False, False],
        [ True, False, False],
        [ True,  True, False]])


In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab),
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [ ]:
l  = ['l1', 'l2', 'l3']
q = ['q1', 'q2', 'q3']
k = ['k1', 'k2', 'k3']
v = ['v1', 'v2', 'v3']

for l, x in zip(l, (q,k,v)):
    print(l, x)
    print("------------------")

l1 ['q1', 'q2', 'q3']
------------------
l2 ['k1', 'k2', 'k3']
------------------
l3 ['v1', 'v2', 'v3']
------------------


In [ ]:
def inference_test():
    test_model = make_model(11, 11, 2)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src)

    for i in range(9):
        out = test_model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()


show_example(run_tests)

In [ ]:
from torch import nn
# from models.layers.scale_dot_product_attention import ScaleDotProductAttention


class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # 1. dot product with weight matrices
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v, mask=mask)

        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        # 5. visualize attention map
        # TODO : we should implement visualization

        return out

    def split(self, tensor):
        """
        split tensor by number of head

        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()

        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        # it is similar with group convolution (split by number of heads)

        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)

        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor


------

## Position-wise Feed-Forward Networks
每个encoder、decoder层中都包含有一个全连接前馈神经网络，在每个层中的位置相同。全连接层只有一个隐藏层，激活函数为$ReLU$。\
计算公式为：$FFN(x) = max(0, xW_1 +b_1)W_2 + b_2$ \
全连接层的输入输出维度为$d_{model}=512$，隐藏层的维度为：$d_{ff}=2048$

### 下面是全连接层的代码实现


In [ ]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## Add & Norm
在每个encoder、decoder层中的每个子层后面，都需要经过残差连接和层归一化。
如下图所示：
<p align="center">
    <img src="imgs/add/Add_Norm_position.png" width="30%">
</p>

### Add 残差连接
残差连接就是将层的输出加上输入，计算公式为：
<p align="center">
    <img src="imgs/add/residual_connect.png" width="30%">
</p>

### LayerNorm 层归一化
层归一化将每一层神经元的输入都转成均值方差
LN 主要用于 NLP 领域，它对每个 token 的特征向量进行归一化计算。设某个 token 的特征向量为 $\pmb{x}\in \mathbb{R}^d$，LN 运算如下  
$\mathrm{LN}(\mathbf{x})=\boldsymbol{\gamma} \odot \frac{\mathbf{x}-\hat{\boldsymbol{\mu}}}{\hat{\boldsymbol{\sigma}}}+\boldsymbol{\beta}$

.其中 $\odot$ 表示按位置乘， **$\pmb{\gamma}, \pmb{\beta}\in \mathbb{R}^d $ 和 是 `拉伸参数scale` 和 `偏移参数shift`，代表着把第$i$个特征的 batch 分布的均值和方差移动到$\beta^i, \gamma^i, \pmb{\gamma}$ 和 $\pmb{\beta}$ 是需要与其他模型参数一起学习的参数**。$\hat{\boldsymbol{\mu}}$ ​ 和 $\hat{\boldsymbol{\sigma}} $ 表示特征向量所有元素的均值和方差，如下计算  
$\hat{\boldsymbol{\mu}}=\frac{1}{d} \sum_{x^i \in \mathbf{x}} x^i$

    
$\hat{\boldsymbol{\sigma}}^{2}=\frac{1}{d} \sum_{x^i \in \mathbf{x}}\left(x^i-\hat{\boldsymbol{\mu}}\right)^{2}+\epsilon$
    
注意我们在方差估计值中添加一个小的常量 $\epsilon$ ，以确保我们永远不会尝试除以零
    
给定一个包含有$L$个token的句子，每个token表示出$d$维的向量，LayerNorm要对每个token的向量进行归一化，共进行$L$次归一化计算，如下图所示:
<p align="center">
    <img src="imgs/add/layer_norm_map.png" width="40%">
</p>

<p align="center">
    <img src="imgs/add/Add_Norm_position_ed.png" width="30%">
</p>

### 残差连接和层归一化的代码实现

In [ ]:
import torch
from torch import nn
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta
        return out

## Cross Attention Mechanism 交叉注意力机制
再回到下面这张Encoder-Decoder模型图中，可以看到，除了Encoder部分的多头注意力外，Encoder部分的输出，是输出到了Decoder的一个注意力子层中。
<p align="center">
    <img src="imgs/decoder/encoder_deconder_cross_attention_o.png" width="30%">
</p>
在Encoder-Decoder模型中，Encoder先处理输入序列，然后最顶部的Encoder的输出，被转换成一组注意力向量$K$，$V$。输出的$K,V$将转递给Decoder中的每一个decoder层中交叉注意力模块使用，**这有助于解码器专注于输入序列中的适当位置**。所以在每一个decoder层中，交叉注意力的$K,V$来自于Encoder，$Q$来自于**Mask Self-Attention**掩码注意力机制。

如下图，需要强调三点：
1. Encoder输出的是$K,V$
2. 只有最后一个encoder层的输出转给decoder
3. 是每一个decoder层都会收到来自encoder的$K,V$
<p align="center">
    <img src="imgs/decoder/encoder_deconder_cross_attention_multied.png" width="30%">
</p>
<p align="center">
    <img src="https://jalammar.github.io/images/t/transformer_decoding_1.gif" width="45%">
    <img src="https://jalammar.github.io/images/t/transformer_decoding_2.gif" width="45%">
</p>


In [ ]:
s = "<strong>{H1_title_content}<mpchecktext contenteditable"
print(s.format(H1_title_content="AAAAA"))

<strong>AAAAA<mpchecktext contenteditable


## Masked-Self-Attention 掩码注意力机制
Decoder中的Self-Attention与编码器中的不同。因为Decoder的目的是基于前文预测下一个词，所以是不能让Decoder参考待预测词后文的，这里的注意力只能看到待预测词之前的文本。这就靠掩码注意力机制来实现，将待预测词之后的位置全部mask掉，设置成$-inf$，再进行Self-Attention计算，计算方式与Encoder中的一致。
如下图所示：
<p align="center">
    <img src="imgs/decoder/mask_self_attention.png" width="50%">
</p>

其中![](https://www.zhihu.com/equation?tex=pos)表示位置，

## Positional Encoding 位置编码
注意力机制能够学习到一个词与其他词的联系，但是无法学习到词与词之间的位置关系。所以，我们需要在Embedding层注入相对或绝对的位置关系。如下图所示，一个token除了编码成一个embedding向量外，还需要构建一个维度相同的$Positional Encoding$向量，将两个向量相加作为Encoder的输入。加入了位置编码后，再映射到$Q/K/V$向量中，在进行Self-Attention计算时，就会考虑到距离关系。
<p align="center">
    <img src="imgs/position/position_encoding_ex.png" width="50%">
</p>

位置编码的计算公式维：

$PE_{(pos, 2i)} = \sin(pos/10000^{2i/d_{model}})$ 

$PE_{(pos, 2i+1)} = \cos(pos/10000^{2i/d_{model}})$ 

其中$pos$表示位置，也就是第几个token，i表示这个token向量的第i维度，偶数位用$\sin$，奇数位用$\cos$。按照论文中所述，选取这个函数作为位置编码是因为这个函数允许模型相对容易的学习到位置关系，因为对于$pos$位置的token，相对于$pos$的任何偏移量$k$的位置信息，$PE_{pos+k}$都可以表示成相对于$PE_{pos}$的线性函数。并且，允许模型在推理阶段处理更长的序列长度。

<p align="center">
    <img src="imgs/position/position_encoding_example.png" width="50%">
</p>

需要注意：**Encoder和Decoder的输入都要添加位置编码信息**

## Positional Encoding 位置编码
注意力机制能够学习到一个词与其他词的联系，但是无法学习到词与词之间的位置关系。所以，我们需要在Embedding层注入相对或绝对的位置关系。如下图所示，一个token除了编码成一个embedding向量外，还需要构建一个维度相同的![](https://www.zhihu.com/equation?tex=Positional%20Encoding)向量，将两个向量相加作为Encoder的输入。加入了位置编码后，再映射到![](https://www.zhihu.com/equation?tex=Q/K/V)向量中，在进行Self-Attention计算时，就会考虑到距离关系。
<p align="center">
    <img src="imgs/position/position_encoding_ex.png" width="50%">
</p>

位置编码的计算公式维：

![](https://www.zhihu.com/equation?tex=PE_%7B%28pos%2C%202i%29%7D%20%3D%20%5Csin%28pos/10000%5E%7B2i/d_%7Bmodel%7D%7D%29) 

![](https://www.zhihu.com/equation?tex=PE_%7B%28pos%2C%202i%2B1%29%7D%20%3D%20%5Ccos%28pos/10000%5E%7B2i/d_%7Bmodel%7D%7D%29) 

其中![](https://www.zhihu.com/equation?tex=pos)表示位置，也就是第几个token，i表示这个token向量的第i维度，偶数位用![](https://www.zhihu.com/equation?tex=%5Csin)，奇数位用![](https://www.zhihu.com/equation?tex=%5Ccos)。按照论文中所述，选取这个函数作为位置编码是因为这个函数允许模型相对容易的学习到位置关系，因为对于![](https://www.zhihu.com/equation?tex=pos)位置的token，相对于![](https://www.zhihu.com/equation?tex=pos)的任何偏移量![](https://www.zhihu.com/equation?tex=k)的位置信息，![](https://www.zhihu.com/equation?tex=PE_%7Bpos%2Bk%7D)都可以表示成相对于![](https://www.zhihu.com/equation?tex=PE_%7Bpos%7D)的线性函数。并且，允许模型在推理阶段处理更长的序列长度。

<p align="center" >
    <img src="imgs/position/position_encoding_example.png" width="50%">
    <text>asdadad<text>
</p>

需要注意：**Encoder和Decoder的输入都要添加位置编码信息**

## The Final Linear and Softmax Layer
Decoder最后输出的是一个浮点数向量，还需要将输出的向量转换成一个词。这一步就是通过一个线性变换层和Softmax层来实现的。

### 线性变换层
就是一个简单的全连接神经网络，能够将解码器生成的向量映射到一个更大的向量，成为$logits$向量。这个更大的向量维度与词表大小相同，也就是需要将Decoder的输出，映射成词表中每一个词的得分。假设模型从训练数据中学习到的词表为10000个词，那么$logits$向量的维度$d_{logits}=10000$。向量中的每个元素代表一个词的得分。

### Softmax
Softmax层就是将$logits$向量中的分数转换成概率，0-1之间的概率，并且所有概率加起来为1，然后概率最大的词作为最终的输出。

<p align="center">
    <img src="imgs/position/final_linner_softmax_layer.png" width="50%">
</p>


 ## 参考文献

 [The Illustrated Transformer](https://jalammar.github.io/illustrated-transformer/)

![alt text](image.png)